In [1]:
import re
import numpy as np

In [2]:
def read_alignment(filename: str = 'test1.txt') -> list:
    '''
    Read in sequence alignment(s). 
    Entire file is line-by-line into a list object and stripped of whitespace.
    List will have representation of some row-wise separator in case of multiple alignments.
    
    ARGS
    -----
    filename: name of file
    
    RETURNS
    --------
    list of alignments separated by any separator (if applicable).
    '''
    with open(filename, 'r') as file:
        return [elem.strip() for elem in file.readlines()]

def columnarize(alignments: list) -> list:
    '''
    Separate alignments in a 1D iterable into 2D (nested list); one row per alignment.
    
    ARGS
    -----
    alignments: list of alignment(s) to separate
    
    RETURNS
    -------
    Nested list of separated alignments
    '''
    nested_list = list()
    temp_list = list()

    for i in range(len(alignments)):
        # If element is part of an alignment
        if re.match('^[A-Z]+|^-|^[a-z]+', alignments[i]):
            # add to a temporary alignment
            temp_list.append(alignments[i])
        # If element is a separator
        else:
            try:
                # add full alignment to a list of alignment(s)
                nested_list.append(temp_list)
                # reset temp alignment in case of more alignments
                temp_list = list()
            except:
                break
    if temp_list:
        nested_list.append(temp_list)
    return nested_list

In [5]:
class Alignment:
    '''
    
    Alignment class helps viewing the each alignment as an individual entity and simplifies 
    the thought process of computing scores and comparing alignments
    
    Class Alignment requires an alignment and score_set corresponding to the possible characters
    in the alignment and calculate number of sequences in the alignment (self.__len) and length of 
    sequences (self.__len_seq) which are useful for debugging.
    
    Class Alignment automatically calculates the entropy for an alignment and has some 
    comparison operators overloaded for comparing entropy scores.

    
    '''
    def __init__(self, alignment, score_set, order = None):
        assert not order or set(order) == score_set, f'order and score_set must contain the same values'
        assert type(order) == list or type(order) == None, f'order argument expected list object; got {type(order)}'
        
        self.alignment = alignment                             # Storing the alignment (need to call alignment fn)
        self.__len = self.__len__()                            # Number of sequences
        self.__len_seq = self.__check_seq_len(alignment)       # Sequence length
        self.__score_set = score_set                           # Score set (could just find distinct chars in alignmnt)
        self.__order = order
        self.T = self.__T()                                    # Transposing alignment for columnarization
        self.__column_counters = self.__alignment_scores()     # Counter for each distinct char for each sequence
        self.profile = \
        self.__get_profile(order = self.__order)               # Alignment profile
        self.entropy = self.__get_entropy()                    # Entropy
        
    
    ''' Accessor functions '''
    def get_len(self):
        return self.__len
    def get_seq_len(self):
        return self.__len_seq
    def get_score_set(self):
        assert self.__score_set is not None, 'score_set is undefined in class initialization'
        return self.__score_set

    ''' Mutator functions '''
    def set_score_set(self, score_set):
        assert type(score_set) == set, f'set_score_set expected {type(set())}, got {type(score_set)}'
        self.__score_set = score_set
        
    ''' Utility functions'''
    def display_profile(self):
        profile_arr = np.array(self.profile).T
        for i in range(profile_arr.shape[0]):
            print(f'{self.__order[i]}', end = ' ')
            for j in range(profile_arr.shape[1]):
                print(profile_arr[i][j], end = ' ')
            print()
        
    ''' Hidden utility functions '''
    def __get_profile(self, order = None):
        '''order sets the profile character order'''
        # Returns a transposed view of the alignment
        if order:
            return [\
                    [self.__column_counters[i][elem]\
                     for elem in order] for i in range(len(self.__column_counters))]
        
        if not order:
            return [\
                    [self.__column_counters[i][elem]\
                     for elem in sorted(self.__score_set)] for i in range(len(self.__column_counters))]
    def __get_entropy(self):
        ZERO_ENTROPY = [0,1]
        entropy = 0
        for column in self.profile:
            for elem in column:
                scaled_elem = elem / sum(column)
                if not scaled_elem in ZERO_ENTROPY:
                    frequency = elem / sum(column)
                    entropy += frequency * np.log2(frequency)
        return round(-entropy, 3)
    
    # Transpose
    def __T(self):
        self.T = list()
        temp_str = ''
        for col in range(self.__len_seq):
            for row in range(self.__len):
                temp_str += self.alignment[row][col]
            self.T.append(temp_str)
            temp_str = ''
        return self.T
    def __alignment_scores(self):
        from collections import Counter
        # Column counts based on column contents
        column_counters = [Counter(elem) for elem in self.T]
        # Column counts including 0-values 
        for column in column_counters:
            for elem in list(self.__score_set):
                if not column[elem]:
                    column[elem] = 0
        ##################################
        return column_counters
    
    def __check_seq_len(self, alignment):
        l = len(alignment[0]) # length of the first element of alignment
        for elem in alignment:
            if len(elem) != l:
                raise Exception('Alignment has sequences of different lengths')
            else:
                return l
            
    def __lt__(self, alignment_object: object) -> bool:
        ''' Overloading the less than (<) operator '''
        return self.entropy < alignment_object.entropy
    
    def __gr__(self, alignment_object: object) -> bool:
        ''' Overloading the greater than (>) operator '''
        return self.entropy > alignment_object.entropy
    
    def __eq__(self, alignment_object: object) -> bool:
        ''' Overloading the greater than (>) operator '''
        return self.entropy == alignment_object.entropy
    
    def __ne__(self, alignment_object: object) -> bool:
        ''' Overloading the greater than (>) operator '''
        return self.entropy != alignment_object.entropy
    

    def __str__(self):
        ''' 
        print() on alignment object returns the alignment
        '''
        return str(self.alignment)
    
    def __repr__(self):
        return str(self.alignment)
    
    def __len__(self):
        '''
        len() on alignment object returns the length of the list object storing the alignment
        '''
        return len(self.alignment)

In [6]:
def display_alignment_comparison():
    pass

def main():
    NUCLEOTIDES = {'G', 'C', 'A', 'T', '-'}
    ORDER = ['A', 'T', 'G', 'C', '-']
    test_files = [f'test{i+1}.txt' for i in range(3)]
    test_alignments = [read_alignment(file) for file in test_files]
    split_alignments = [columnarize(alignments) for alignments in test_alignments]
    alignment_pair_objs = [[Alignment(alignment, NUCLEOTIDES, ORDER)\
                           for alignment in almnt_pair] for almnt_pair in split_alignments]
    
    for pair in alignment_pair_objs:
        for i in range(len(pair)):
            print(f'Alignment {i+ 1} profile:', end = '\n\n')
            pair[i].display_profile()
            print()
            print(f'Entropy score is {pair[i].entropy}', end = '\n\n')
        if pair[0] < pair[1]:
            print('Alignment 1 is better')
        elif pair[1] < pair[0]:
            print('Alignment 2 is better')
        else:
            print('Alignments have the same entropy')
        print()
        print('####################################')
        print('####################################')
        print()
            

main()

Alignment 1 profile:

A 4 1 1 
T 0 0 1 
G 0 0 1 
C 0 3 1 
- 0 0 0 

Entropy score is 2.811

Alignment 2 profile:

A 4 1 1 0 
T 0 0 0 1 
G 0 0 1 0 
C 0 1 2 1 
- 0 2 0 2 

Entropy score is 4.5

Alignment 1 is better

####################################
####################################

Alignment 1 profile:

A 4 0 0 
T 0 0 0 
G 0 1 1 
C 0 3 1 
- 0 0 2 

Entropy score is 2.311

Alignment 2 profile:

A 4 0 0 
T 0 0 0 
G 0 0 2 
C 0 3 1 
- 0 1 1 

Entropy score is 2.311

Alignments have the same entropy

####################################
####################################

Alignment 1 profile:

A 0 0 0 2 2 
T 0 0 1 0 0 
G 3 2 2 1 0 
C 0 0 0 0 1 
- 0 1 0 0 0 

Entropy score is 3.673

Alignment 2 profile:

A 0 0 0 2 2 
T 0 0 1 0 0 
G 3 3 2 0 0 
C 0 0 0 1 0 
- 0 0 0 0 1 

Entropy score is 2.755

Alignment 2 is better

####################################
####################################

